<img src="../images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>

# Feedback analysis using radiative kernels

---

## Overview

This notebook details all of the steps required to perform a radiative kernel analysis.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Loading CMIP6 data | Necessary | |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary |  |

- **Time to learn**: 60 minutes

---

## Imports

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import intake

## Load in required data

In [ ]:
cat_url = 'https://storage.googleapis.com/cmip6/pangeo-cmip6.json'
col = intake.open_esm_datastore(cat_url)

The fields (and CMIP names) required to calculate each feedback are:
- Albedo: upwelling and downwelling SW radiation at the surface (`rsus` and `rsds`)
- Temperature (Planck and lapse rate): air temperature (`ta`) and surface temperature (`ts`)
- Water vapor: specific humidity (`hus`) and air temperature
- SW CRE: Net SW radiation at TOA (down `rsdt` minus up `rsut`) and clear-sky versions (down, which is the same, minus up `rsutcs`)
- LW CRE: Net LW radiation at TOA (`rlut`) and the clear-sky version (`rlutcs`)

The cloud feedbacks also require the results from the other feedbacks to correct for noncloud contributions to the CREs.

In [ ]:
cat = col.search(activity_id='CMIP', experiment_id=['piControl', 'abrupt-4xCO2'], table_id='Amon', source_id='CESM2', 
                 variable_id=['rsus', 'rsds', 'ta', 'ts', 'hus', 'rsdt', 'rsut', 'rsutcs', 'rlut', 'rlutcs'])
cat.df

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True})

In [ ]:
ctrl = dset_dict['CMIP.NCAR.CESM2.piControl.Amon.gn']

In [ ]:
ctrl

In [ ]:
pert = dset_dict['CMIP.NCAR.CESM2.abrupt-4xCO2.Amon.gn']

In [ ]:
pert

## Prepare data for analysis

In [ ]:
def global_average(data):
    weights = np.cos(np.deg2rad(data.lat))
    data_weighted = data.weighted(weights)
    return data_weighted.mean(dim=['lat', 'lon'], skipna=True)

In [ ]:
gmst_ctrl = global_average(ctrl.ts.rolling(time=12, center=True).mean())

In [ ]:
gmst_ctrl.sel(time=slice('0001', '0101')).plot()

In [ ]:
gmst_pert = global_average(pert.ts.rolling(time=12, center=True).mean())

In [ ]:
gmst_pert.sel(time=slice('0001', '0101')).plot()

In [ ]:
ctrl_state = ctrl.sel(time=slice('1100', '1200')).mean(dim='time')
pert_state = pert.sel(time=slice('0899', '0999')).mean(dim='time')

In [ ]:
ctrl_SWCRE = (-ctrl_state.rsut + ctrl_state.rsutcs)
pert_SWCRE = (-pert_state.rsut + pert_state.rsutcs)
dCRE = pert_SWCRE - ctrl_SWCRE

In [ ]:
dCRE.plot()

## Calculate feedbacks

### Albedo

### Temperature

#### Planck

#### Lapse rate

### Water vapor

#### Shortwave water vapor

#### Longwave water vapor

### Cloud

#### Cloud radiative effect (CRE)

#### Cloud feedback adjustments

### Net

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!